In [1]:
import torch
import json

In [2]:
with open("hierarchy_flat.json") as f:
    hierarchy_flat = json.load(f)

In [3]:
with open(f"concepts.json") as f:
    hierarchy = json.load(f)

In [4]:
reverse_flat_hierarchy = {"level1": {}, "level2": {}, "level3": {}}
for _level in hierarchy_flat.keys():
    if _level == "level4":
        continue
    reverse_flat_hierarchy[_level] = {}
    for key, value in hierarchy_flat[_level].items():
        for val in value:
            reverse_flat_hierarchy[_level][val] = key

In [5]:
levels_none_index = {'level1': -1, 'level2': -1, 'level3': -1, 'level4': -1}
for key in hierarchy_flat:
    if key != "level1":
        levels_none_index[key] = len(hierarchy_flat[key])

In [6]:
levels_none_index

{'level1': -1, 'level2': 157, 'level3': 62, 'level4': 7}

In [7]:
file = "data/test_logits.pt"
data = torch.load(file)

In [8]:
print(data.keys())
print(data['levels1'].keys())

dict_keys(['levels1', 'levels2', 'levels3', 'levels4'])
dict_keys(['logit', 'labels'])


In [9]:
key_map = {'levels1': 'level1', 'levels2': 'level2', 'levels3': 'level3', 'levels4': 'level4'}

In [10]:
for level in data.keys():
    data[level]['preds'] = data[level]['logit'].argmax(-1)
    data[level]['labels'][data[level]['labels'] == -1] = levels_none_index[key_map[level]]

In [11]:
back_step = {'levels3': 'levels2', 'levels4': 'levels3'}
for level in data.keys():
    if level in back_step:
        back_step_key = back_step[level]
        back_step_none = levels_none_index[key_map[back_step_key]]
        current_step_none = levels_none_index[key_map[level]]
        data[level]['preds'][data[back_step_key]['preds'] == back_step_none] = current_step_none

In [12]:
reverse_flat_hierarchy_numbered = {'level1': {}, 'level2': {}, 'level3': {}}
for level in reverse_flat_hierarchy_numbered:
    for key, val in reverse_flat_hierarchy[level].items():
        try:
            if level == 'level1':
                key_space = hierarchy['level2']
                val_space = hierarchy['level1']
            elif level == 'level2':
                key_space = hierarchy['level3']
                val_space = hierarchy['level2']
            elif level == 'level3':
                key_space = hierarchy['level4']
                val_space = hierarchy['level3']
            key_number = key_space.index(key)
            val_number = val_space.index(val)
            reverse_flat_hierarchy_numbered[level][key_number] = val_number
        except:
            print(key, val)
            print(level)
            print(key_space)
            print(val_space)
            raise

In [13]:
map_back_step = {'level3': 'level2', 'level4': 'level3', 'level2': 'level1'}
map_back_step_original = {'levels3': 'levels2', 'levels4': 'levels3', 'levels2': 'levels1'}
for level in ['levels4', 'levels3', 'levels2', 'levels1']:
    level_maped = key_map[level] ### levels4 --> level4
    if level_maped in map_back_step:
        level_back_original = map_back_step_original[level] ### levels4 --> levels3
        maped_back_step_level = map_back_step[level_maped] ### level4 --> level3
        for key in hierarchy_flat[level_maped]:
            if key in reverse_flat_hierarchy[maped_back_step_level]:
                key_number = hierarchy[level_maped].index(key)
                val_number = reverse_flat_hierarchy_numbered[maped_back_step_level][key_number]
                data[level_back_original]['preds'][data[level]['preds'] == key_number] = val_number
                
                
                

In [14]:
print(levels_none_index)
for k in range(0, 10):
    level1 = data['levels1']['preds'][k].item()
    level2 = data['levels2']['preds'][k].item()
    level3 = data['levels3']['preds'][k].item()
    level4 = data['levels4']['preds'][k].item()
    
    original_level1 = data['levels1']['logit'].argmax(-1)[k].item()
    original_level2 = data['levels2']['logit'].argmax(-1)[k].item()
    original_level3 = data['levels3']['logit'].argmax(-1)[k].item()
    original_level4 = data['levels4']['logit'].argmax(-1)[k].item()
    
    if (torch.tensor([level1, level2, level3, level4]) != torch.tensor([original_level1, original_level2, original_level3, original_level4])).sum() != 0:
        parents = []
        if level4 == len(hierarchy_flat['level4']):
            parents.insert(0, level3)
        else:
            parent_key = reverse_flat_hierarchy_numbered['level3'][level4]
            parents.insert(0, parent_key)

        if level3 == len(hierarchy_flat['level3']):
            parents.insert(0, level2)
        else:
            parent_key = reverse_flat_hierarchy_numbered['level2'][level3]
            parents.insert(0, parent_key)

        if level2 == len(hierarchy_flat['level2']):
            parents.insert(0, level1)
        else:
            parent_key = reverse_flat_hierarchy_numbered['level1'][level2]
            parents.insert(0, parent_key)

        print(f" {k} updated preds are: {level1}, {level2}, {level3}, {level4}")
        print(f"original preds were: {original_level1}, {original_level2}, {original_level3}, {original_level4}")
        print(f"parent are {parents} \n")
    
    

{'level1': -1, 'level2': 157, 'level3': 62, 'level4': 7}


In [15]:
print(reverse_flat_hierarchy_numbered['level1'][131])
print(hierarchy['level2'][131])
print(hierarchy['level1'][reverse_flat_hierarchy_numbered['level1'][131]])

35
ball
toy


In [16]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

In [17]:
l1_pr, l1_rec, l1_f1, l1_sup = precision_recall_fscore_support(
    np.array(data['levels1']['logit'].argmax(-1)), 
    np.array(data['levels1']['labels'])
)

l2_pr, l2_rec, l2_f1, l2_sup = precision_recall_fscore_support(
    np.array(data['levels2']['logit'].argmax(-1)), 
    np.array(data['levels2']['labels'])
)

l3_pr, l3_rec, l3_f1, l3_sup = precision_recall_fscore_support(
    np.array(data['levels3']['logit'].argmax(-1)), 
    np.array(data['levels3']['labels'])
)

l4_pr, l4_rec, l4_f1, l4_sup = precision_recall_fscore_support(
    np.array(data['levels4']['logit'].argmax(-1)), 
    np.array(data['levels4']['labels'])
)



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
def calculate_f1_score(preds, labels, level_key, key_original, hierarchy, data):
    l1_pr, l1_rec, l1_f1, l1_sup = precision_recall_fscore_support(
        np.array(preds), 
        np.array(labels)
    )
    
    f1_averaged = 0
    total_instances = 0
    for i, key in enumerate(hierarchy[level_key]):
        key_number = hierarchy[level_key].index(key)
        support_val = (data[key_original]['labels'] == key_number).sum().item()
        # print(l1_f1[i], key, l1_sup[i], support_val)
        f1_averaged += support_val * l1_f1[i]
        total_instances += support_val
    f1_averaged = f1_averaged / total_instances
    print(level_key, f1_averaged, total_instances)
    return f1_averaged, total_instances
    

In [30]:
f1_total = 0
f1_instances = 0
for key, key_org in zip(
    ['level1', 'level2', 'level3', 'level4'], 
    ['levels1', 'levels2', 'levels3', 'levels4']):
    f1, instances = calculate_f1_score(
        data[key_org]['logit'].argmax(-1),
        data[key_org]['labels'],
        key, key_org, hierarchy, data)
    f1_total += f1 * instances
    f1_instances += instances
    
print(f1_total / f1_instances)

    
f1_total = 0
f1_instances = 0
for key, key_org in zip(
    ['level1', 'level2', 'level3', 'level4'], 
    ['levels1', 'levels2', 'levels3', 'levels4']):
    f1, instances = calculate_f1_score(
        data[key_org]['preds'],
        data[key_org]['labels'],
        key, key_org, hierarchy, data)
    f1_total += f1 * instances
    f1_instances += instances
    
print(f1_total / f1_instances)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


level1 0.56733774088159 376332
level2 0.5445647981840609 199265


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


level3 0.43429322770035084 60746
level4 0.1768860196386531 1783
0.5464704704000442


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


level1 0.5581853041826627 376332
level2 0.5317166891057793 199265


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


level3 0.43445783653106423 60746
level4 0.24184303151155342 1783
0.5372580020425968


0

tensor(611)